In [1]:
# Imports the Google Cloud client library

import bq_helper
from bq_helper import BigQueryHelper

import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = str('/Users/my_macbook/Projects/Project_Capstone/CapstoneGStoken.json')


from plotly import __version__

from plotly.offline import init_notebook_mode, iplot

import plotly.graph_objs as go
import matplotlib.pyplot as plt
import plotly.plotly as py
from plotly import tools


import pandas as pd
init_notebook_mode(connected=True)

# https://github.com/SohierDane/BigQuery_Helper
from bq_helper import BigQueryHelper
medicare = bq_helper.BigQueryHelper(active_project="bigquery-public-data", dataset_name="cms_medicare")
bq_assistant = BigQueryHelper("bigquery-public-data", "cms_medicare")

bq_assistant.list_tables()





['home_health_agencies_2013',
 'home_health_agencies_2014',
 'hospice_providers_2014',
 'hospital_general_info',
 'inpatient_charges_2011',
 'inpatient_charges_2012',
 'inpatient_charges_2013',
 'inpatient_charges_2014',
 'inpatient_charges_2015',
 'nursing_facilities_2013',
 'nursing_facilities_2014',
 'outpatient_charges_2011',
 'outpatient_charges_2012',
 'outpatient_charges_2013',
 'outpatient_charges_2014',
 'outpatient_charges_2015',
 'part_d_prescriber_2014',
 'physicians_and_other_supplier_2012',
 'physicians_and_other_supplier_2013',
 'physicians_and_other_supplier_2014',
 'physicians_and_other_supplier_2015',
 'referring_durable_medical_equip_2013',
 'referring_durable_medical_equip_2014']

In [2]:
query1 = """SELECT 

nppes_provider_state AS state,
ROUND(SUM(total_claim_count) / 1e6) AS total_claim_count_millions
FROM
  `bigquery-public-data.medicare.part_d_prescriber_2014`
GROUP BY
  state
ORDER BY
  total_claim_count_millions DESC;"""
total_count_query = medicare.query_to_pandas_safe(query1)
total_count_query.head(10)

,state,total_claim_count_millions
0,CA,116.0
1,FL,91.0
2,NY,80.0
3,TX,76.0
4,PA,63.0
5,OH,53.0
6,NC,46.0
7,IL,43.0
8,GA,39.0
9,MI,39.0


In [3]:
#scl = [[0.0, 'rgb(248,255,206)'],[0.2, 'rgb(203,255,205)'],[0.4, 'rgb(155,255,164)'], [0.6, 'rgb(79,255,178)'],[0.8, 'rgb(15,183,132)'], [1, '#008059']]
scl = [[0.0, 'rgb(255,248,206)'],[0.2, 'rgb(255,203,205)'],[0.4, 'rgb(255,155,164)'], [0.6, 'rgb(255,79,178)'],[0.8, 'rgb(183,15,132)'], [1, '#008059']]
data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = total_count_query.state,
        z = total_count_query.total_claim_count_millions,
        locationmode = 'USA-states',
        text = total_count_query.state,
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Total Claims in Different States")
        )
       ]

layout = dict(
        title = 'Total Claims in Different States',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
iplot( fig, filename='d3-cloropleth-map' )

In [4]:
query2 = """SELECT
drug_name AS drug,
  ROUND(SUM(total_drug_cost) / 1e6) AS total_drug_count_millions
FROM
  `bigquery-public-data.medicare.part_d_prescriber_2014`
GROUP BY
  drug
ORDER BY
  total_drug_count_millions DESC;"""
total_drug_query = medicare.query_to_pandas_safe(query2)
total_drug_query.head(10)

,drug,total_drug_count_millions
0,SOVALDI,2486.0
1,NEXIUM,2422.0
2,CRESTOR,2396.0
3,ABILIFY,2247.0
4,ADVAIR DISKUS,2065.0
5,SPIRIVA,1957.0
6,LANTUS SOLOSTAR,1806.0
7,JANUVIA,1592.0
8,LANTUS,1512.0
9,REVLIMID,1501.0


In [5]:
tempdf = total_drug_query.head(30)
y = list(reversed(list(tempdf.drug)))
x = list(reversed(list(tempdf.total_drug_count_millions)))
trace1 = go.Bar(x=x, y=y, orientation="h", marker=dict(color='#f79284'),)
layout = dict(height=400, width=700, margin=dict(l=400), title='Drug Ranking per total cost')
data = [trace1]

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='stacked-bar')

In [6]:
query1 = """SELECT
  COUNT(DISTINCT npi) AS npi_id,
  nppes_provider_state AS state,
  ROUND(SUM(total_claim_count) / 1e6) AS total_claim_count_millions,
  ROUND(SUM(total_day_supply) / 1e6) AS total_day_supply_millions,
  ROUND(SUM(total_drug_cost) / 1e6) AS total_drug_cost_millions
FROM
  `bigquery-public-data.cms_medicare.part_d_prescriber_2014`
GROUP BY
  state
ORDER BY
  total_claim_count_millions DESC;
        """
total_state__claim_drug_query = medicare.query_to_pandas_safe(query1)
total_state__claim_drug_query.head(10)



,npi_id,state,total_claim_count_millions,total_day_supply_millions,total_drug_cost_millions
0,86588,CA,116.0,4935.0,9634.0
1,49875,FL,91.0,3770.0,6970.0
2,63752,NY,80.0,3150.0,7522.0
3,50854,TX,76.0,3105.0,6463.0
4,41440,PA,63.0,2402.0,4842.0
5,33064,OH,53.0,2181.0,4051.0
6,26876,NC,46.0,1706.0,3358.0
7,32415,IL,43.0,1724.0,3121.0
8,20729,GA,39.0,1403.0,2788.0
9,29897,MI,39.0,1797.0,3194.0


In [7]:
total_state__claim_drug_query.count

<bound method DataFrame.count of     npi_id state  total_claim_count_millions  total_day_supply_millions  \
0    86588    CA                       116.0                     4935.0   
1    49875    FL                        91.0                     3770.0   
2    63752    NY                        80.0                     3150.0   
3    50854    TX                        76.0                     3105.0   
4    41440    PA                        63.0                     2402.0   
5    33064    OH                        53.0                     2181.0   
6    26876    NC                        46.0                     1706.0   
7    32415    IL                        43.0                     1724.0   
8    20729    GA                        39.0                     1403.0   
9    29897    MI                        39.0                     1797.0   
10   18826    TN                        35.0                     1331.0   
11   16128    MO                        31.0                     11

In [18]:
trace1 = go.Scatter(
    x=total_state__claim_drug_query.total_day_supply_millions,
    y=total_state__claim_drug_query.total_drug_cost_millions,
    mode='markers',
    text=total_state__claim_drug_query.state,
    marker=dict(
        color=list(reversed([i*11 for i in range(200,251)])),
        size=total_state__claim_drug_query.total_claim_count_millions,
        line=dict(
            width=2
        ),
    )
)
data = [trace1]

layout = go.Layout(
    title='Total claim count | Total Drug Cost | Total Day Supply for every state',
    xaxis=dict(
        title='Total Day Supply',
        gridcolor='rgb(255, 255, 255)',
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    yaxis=dict(
        title='Total Drug Cost',
        gridcolor='rgb(255, 255, 255)',
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='scatter-colorscale')


In [10]:
query1 = """SELECT
 # COUNT(DISTINCT npi) AS NPI,
  generic_name AS drug_name,
  ROUND(SUM(total_claim_count) / 1e6) AS total_claim_count_millions,
  ROUND(SUM(total_day_supply) / 1e6) AS total_day_supply_millions,
  ROUND(SUM(total_drug_cost) / 1e6) AS total_drug_cost_millions,
  MAX(total_claim_count) AS max_claim,
  MAX(total_day_supply)  AS max_day,
  MAX(total_drug_cost) AS  max_cost,
  VARIANCE(total_claim_count) AS var_claim,
  VARIANCE(total_day_supply) AS var_day,
  VARIANCE(total_drug_cost) AS var_cost

FROM
  `bigquery-public-data.cms_medicare.part_d_prescriber_2014`
GROUP BY
  drug_name
ORDER BY
  total_claim_count_millions DESC;
        """
most_pres_drug = medicare.query_to_pandas_safe(query1)
most_pres_drug.head(10)


,drug_name,total_claim_count_millions,total_day_supply_millions,total_drug_cost_millions,max_claim,max_day,max_cost,var_claim,var_day,var_cost
0,LEVOTHYROXINE SODIUM,42.0,2154.0,831.0,6498,171754,152571.44,31943.599578,7.133442e+07,1.183979e+07
1,LISINOPRIL,38.0,1954.0,278.0,5923,129872,45180.97,26397.120044,6.687316e+07,1.565043e+06
2,AMLODIPINE BESYLATE,36.0,1774.0,302.0,7516,141792,62784.00,29722.186065,6.275568e+07,2.100734e+06
3,SIMVASTATIN,34.0,1844.0,343.0,3994,165190,53232.74,28002.370681,7.902331e+07,3.092969e+06
4,ATORVASTATIN CALCIUM,32.0,1689.0,748.0,4687,188889,132071.09,24894.929778,6.696477e+07,1.459274e+07
5,HYDROCODONE/ACETAMINOPHEN,32.0,656.0,669.0,7979,237610,253168.34,34554.729556,2.441357e+07,2.095799e+07
6,OMEPRAZOLE,32.0,1487.0,519.0,4884,142942,78243.60,24585.603688,4.730298e+07,6.644458e+06
7,METFORMIN HCL,27.0,1355.0,326.0,2670,98912,116479.46,12610.579027,2.853501e+07,2.984932e+06
8,FUROSEMIDE,26.0,1077.0,133.0,8070,183059,49104.29,27232.708207,2.852193e+07,8.094469e+05
9,GABAPENTIN,21.0,820.0,491.0,5255,136212,140668.81,13596.703217,1.679392e+07,7.771740e+06


In [11]:
tempdf = most_pres_drug.head(50)

trace1 = go.Bar(
    x=tempdf.drug_name,
    y=tempdf.total_claim_count_millions,
    name='Total Claims'
)
trace2 = go.Bar(
    x=tempdf.drug_name,
    y=tempdf.total_day_supply_millions,
    name='Total Day Supply'
)
trace3 = go.Bar(
    x=tempdf.drug_name,
    y=tempdf.total_drug_cost_millions,
    name='Total Drug Cost'
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='group'
    , margin=dict(b=200)
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='grouped-bar')

In [12]:
query1 = """SELECT
  
  specialty_description AS specialty_description,
  ROUND(SUM(total_claim_count) / 1e6) AS total_claim_count_millions,
  ROUND(SUM(total_day_supply) / 1e6) AS total_day_supply_millions,
  ROUND(SUM(total_drug_cost) / 1e6) AS total_drug_cost_millions
FROM
  `bigquery-public-data.cms_medicare.part_d_prescriber_2014`
GROUP BY
  specialty_description
ORDER BY
  total_claim_count_millions DESC;
        """
specialty_query = medicare.query_to_pandas_safe(query1)
specialty_query.head(40)

,specialty_description,total_claim_count_millions,total_day_supply_millions,total_drug_cost_millions
0,Internal Medicine,386.0,16341.0,22985.0
1,Family Practice,361.0,14968.0,18255.0
2,Nurse Practitioner,78.0,2873.0,6403.0
3,Cardiology,67.0,3402.0,4039.0
4,Psychiatry,41.0,1307.0,4192.0
5,Physician Assistant,40.0,1494.0,3020.0
6,General Practice,30.0,1046.0,1425.0
7,Ophthalmology,24.0,816.0,2135.0
8,Neurology,19.0,696.0,5088.0
9,Nephrology,14.0,575.0,1825.0


In [13]:
tempdf = specialty_query.head(40)
trace1 = go.Bar(
    x=tempdf.specialty_description,
    y=tempdf.total_claim_count_millions,
    name='Total Claims'
)
trace2 = go.Bar(
    x=tempdf.specialty_description,
    y=tempdf.total_day_supply_millions,
    name='Total Day Supply'
)
trace3 = go.Bar(
    x=tempdf.specialty_description,
    y=tempdf.total_drug_cost_millions,
    name='Total Drug Cost'
)

data = [trace1, trace2, trace3]
layout = go.Layout(
    barmode='stack'
    , margin=dict(b=200)
)

fig = go.Figure(data=data, layout=layout)
iplot(fig, filename='grouped-bar')



In [14]:
query1 = """SELECT 
A.state, B.drug_name, A.MaxClaimCount from (select state, MAX(total_claim_count_millions) as MaxClaimCount 
from (
SELECT
  nppes_provider_state AS state,
  generic_name AS drug_name,
  SUM(total_claim_count) AS total_claim_count_millions
FROM
  `bigquery-public-data.cms_medicare.part_d_prescriber_2014`
GROUP BY
  state, drug_name
ORDER BY
  total_claim_count_millions DESC) group by state) A INNER JOIN (SELECT
  nppes_provider_state AS state,
  generic_name AS drug_name,
  SUM(total_claim_count) AS total_claim_count_millions
FROM
  `bigquery-public-data.cms_medicare.part_d_prescriber_2014`
GROUP BY
  state, drug_name
ORDER BY
  total_claim_count_millions DESC) B ON A.MaxClaimCount = B.total_claim_count_millions;
"""
state_most_drug = medicare.query_to_pandas_safe(query1)
state_most_drug.head(10)

,state,drug_name,MaxClaimCount
0,AA,"SYRINGE,NEEDLE,INSULN,SAFE,1ML",225
1,AA,GANCICLOVIR,225
2,AA,FLUTICASONE FUROATE,225
3,AA,BUTALBITAL/ACETAMINOPHEN,225
4,AA,ABATACEPT/MALTOSE,225
5,AA,NEVIRAPINE,225
6,AA,"PEN NEEDLE, DIABETIC, SAFETY",225
7,AA,TRIFLUOPERAZINE HCL,225
8,AA,SODIUM BICARBONATE,225
9,AA,DEXTROAMPHETAMINE SULFATE,225


In [15]:
visited = {}
rows = []
for index, row in state_most_drug.iterrows():
    stat = row['state']
    if stat not in visited:
        visited[stat] = 1
        rows.append(row)
newDF = pd.DataFrame(rows).reset_index()
newDF[['state', 'drug_name', 'MaxClaimCount']].head(41)

,state,drug_name,MaxClaimCount
0,AA,"SYRINGE,NEEDLE,INSULN,SAFE,1ML",225
1,AE,GOLIMUMAB,666
2,AK,LISINOPRIL,47684
3,AL,HYDROCODONE/ACETAMINOPHEN,1214487
4,AP,DOXERCALCIFEROL,208
5,AR,HYDROCODONE/ACETAMINOPHEN,587718
6,AS,TELMISARTAN/AMLODIPINE,46
7,AZ,LEVOTHYROXINE SODIUM,866249
8,CA,LEVOTHYROXINE SODIUM,3845087
9,CO,LEVOTHYROXINE SODIUM,644486


In [15]:
a = newDF['drug_name'].value_counts()
labels = a.index
values = a.values
colors = ['lightblue','gray','#eee','#999', '#9f9f']
trace = go.Pie(labels=labels, values=values, hoverinfo='label+percent', 
               textinfo='value', name='Top Drugs Used',
               marker=dict(colors=colors))
layout = dict(title = 'Top Drugs in Each State',
              xaxis= dict(title= 'Drug Name', ticklen= 5,zeroline= False),
              width=800
             )
fig = dict(data = [trace], layout = layout)
iplot(fig)